In [1]:
import random
from datetime import datetime, date
from math import log, sqrt, pi, exp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from scipy.stats import norm
import tensortrade.stochastic as sp

np.set_printoptions(precision=3)
pd.set_option("display.precision", 2)

plt.style.use("ggplot")
plt.rcParams["figure.figsize"] = 12, 8

In [2]:
data = sp.gbm(
    base_price=100,
    base_volume=5,
    start_date="2010-01-01",
    times_to_generate=1000,
    time_frame='1H'
)

data

,open,high,low,close,volume
2010-01-01 00:00:00,100.00,101.31,99.69,99.73,291.28
2010-01-01 01:00:00,99.85,100.81,99.50,99.97,305.83
2010-01-01 02:00:00,100.02,100.69,99.45,99.45,303.25
2010-01-01 03:00:00,99.28,99.66,98.09,98.12,291.54
2010-01-01 04:00:00,98.09,99.06,97.37,98.97,299.11
...,...,...,...,...,...
2010-02-11 11:00:00,235.68,236.70,233.33,235.84,291.70
2010-02-11 12:00:00,235.79,238.39,234.07,238.39,298.44
2010-02-11 13:00:00,237.55,238.68,233.78,234.33,288.24
2010-02-11 14:00:00,234.95,234.95,225.80,226.24,287.53


# RL Sample Code

In [3]:
# env = gym.make('FrozenLake-v0')

# def train(alpha,eps,dis,episodes,min_eps,eps_rate):
#     Q = np.zeros((16,4))
#     for episode in range(episodes):
#         state = env.reset()
#         for t in range(100):
#             p = [1-3/4*eps if x == np.argmax(Q[state]) else eps/4 for x in range(4)]
#             action = np.random.choice(range(4),size=1,p=p)[0]
#             new_state, reward, done, info = env.step(action)
#             Q[state][action] += alpha*(reward+dis*np.max(Q[new_state])-Q[state][action])
#             if done:
#                 break
#             else:
#                 state = new_state
#         if eps > min_eps:
#             eps *= (1-eps_rate)
#     env.close()
#     return Q

# Q = train(alpha=0.1,eps=1.,dis=0.99,episodes=10000,min_eps=0.1,eps_rate=0.001)

# Almgren-Chriss

In [4]:
def impact_perm(nu, gamma, beta):
    """Returns the permenant dollar price impact per unit time

    In paper as :math:`g(\nu)`

    Args:
        nu: rate of trading :math:`\nu = n_k / \tau`.
        gamma: Const. the $ move per share traded
        beta: Const. power law scaling of nu for perm impact.
            Default is .5 for square root rule
    """
    return gamma * nu ** beta


def impact_temp(nu, eps, eta, alpha):
    """Returns the temporary dollar price impact

    In 2005 paper as :math:`h(\nu)`

    Args:
        nu: rate of trading :math:`\nu = n_k / \tau`.
        eps: Const. $ move
        eta: Const. the $ move per trading speed
        alpha: Const. power law scaling of nu for temp impact.
            Deafult if 1 for arbitrage free linear model
    """
    return eps * np.sign(nu) + eta * nu ** alpha


def is_expected(n_t, gamma, eta, eps, tau):
    """The expected implementation shortfall

    Args:
        n_t: array of units executed at each time step where
            ``len(n_t * tau)`` is the time taken to execute
        gamma: Const. the $ move per share traded
        eta: Const. the $ move per trading speed
        eps: Const. the $ move per share traded
        tau: Time step between each element of n_t
    """
    x_k = np.cumsum(n_t[::-1])  # units left to execute
    nu_t = n_t / tau
    return (
        np.sum(tau * x_k * impact_perm(nu_t)) +
        np.sum(n_t * impact_temp(nu_t))
    )


def is_var(n_t, sigma, tau):
    """The vairance of the implementation shortfall

    Args:
        n_t: array of units executed at each time step where
            ``len(n_t * tau)`` is the time taken to execute
        sigma: The vol of underlying securities
        tau: Time step between each element of n_t
    """
    x_k = np.cumsum(n_t[::-1])  # units left to execute
    return sigma**2 + tau * np.dot(x_k.T, x_k)


def is_objective(n_t, risk_tol, gamma, eta, eps, tau, sigma):
    """Almgren-Chriss objective function

    Args:
        n_t: array of units executed at each time step where
            ``len(n_t * tau)`` is the time taken to execute
        gamma: Const. the $ move per share traded
        eta: Const. the $ move per trading speed
        eps: Const. the $ move per share traded
        tau: Time step between each element of n_t
    """

    return (is_expected(n_t, gamma, eta, eps, tau) +
            risk_tol*is_var(n_t, sigma, tau))


def trade_decay_rate(tau, risk_tol, sigma, eta, gamma):
    """Also known as :math:`\kappa` in the paper

    Note:
        :math:`\kappa^{-1}` is the time it takes to
        deplete the portfolio by a factor of :math:`e`
        If :math:`\lambda > 0` the trader will still liquidate the
        position on a time scale `:math:`\kappa^{-1}` so
        :math:`\kappa^{-1}` is the intrinsic time scale of the trade.

    Args:
        tau: Time step between each element of :math:`n_t`
        risk_tol: The risk tolerance
        sigma: volatility of the unit price
        eta: Const. the $ move per trading speed
        gamma: Const. the $ move per share traded
    """
    return np.sqrt(risk_tol*sigma**2 / (eta * (1 + .5*gamma*tau/eta)))


def trading_traj(trading_time, tau, risk_tol, sigma, eta, gamma):
    """Returns the optimal trading trajectory :math:`n_t`
    (allocates the distribution of units over tau)

    Args:
        trading_time: Total time to trade
        tau: Time step between each element of :math:`n_t`
        risk_tol: The risk tolerance
        sigma: volatility of the unit price
        eta: Const. the $ move per trading speed
        gamma: Const. the $ move per share traded
    """
    k = trade_decay_rate(risk_tol, sigma, eta, gamma, tau)
    tj = np.arange(trading_time/tau) * tau
    return 2*np.sinh(.5*k*tau)/np.sinh(k*trading_time)*np.cosh(k*tj*trading_time)

# Black-Scholes

In [5]:
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)
    
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

def call_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm.cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm.cdf(d2(S, K, T, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"


def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
    
def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))

In [7]:
stock = 'SPY'
expiry = '12-18-2022'
strike_price = 370

today = datetime.now()
one_year_ago = today.replace(year=today.year-1)

df = yf.download('SPY', start=one_year_ago, end=today)

df = df.sort_values(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

sigma = np.sqrt(252) * df['returns'].std()
uty = (yf.download('^TNX', start=today.replace(day=today.day-1), end=today)['Close'].iloc[-1])/100
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%m-%d-%Y") - datetime.utcnow()).days / 365

print('The Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma))


print("Implied Volatility: " +
      str(100 * call_implied_volatility(bs_call(lcp, strike_price, t, uty, sigma,), lcp, strike_price, t, uty,)) + " %")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
The Option Price is:  81.5456288251815
